<font color="RED"><h1><b>EXPENSE TRACKER</b></h1></font>

In [3]:
class Expense_Tracker:
    def __init__(self):
        self.use=input("\nEnter your SQL Server UserName : ")
        self.passwo=g.getpass("Enter your SQL Server Password : ")
        self.con=mysql.connector.connect(
            host="localhost",
            user=self.use,
            password=self.passwo,
            database="Expense_Tracker"
        )
        self.c=self.con.cursor()
        self.c.execute("Create table if not exists user_details(Phone_Number bigint, Email_ID text, Name text, Password text)")
        self.c.execute("Create table if not exists income(Income_ID bigint, Income_Source text, Salary_Amount int, Income_Date date)")
        self.con.commit()
    def signup(self):
        def validate_password(password):
            print()
            if len(password) < 8:
                print("Password must be at least 8 characters long.")
                return False
            if not re.search(r"[A-Z]", password):
                print("Password must contain at least one uppercase letter.")
                return False
            if not re.search(r"[a-z]", password):
                print("Password must contain at least one lowercase letter.")
                return False
            if not re.search(r"[0-9]", password):
                print("Password must contain at least one digit.")
                return False
            if not re.search(r"[!@#$%^&*(),.?\":{}|<>]", password):
                print("Password must contain at least one special character.")
                return False
            return password
        while True:
            PN=int(input("\nEnter your Phone Number : "))
            self.c.execute("Select Phone_Number from user_details")
            print()
            phonerecords=[i[0] for i in self.c.fetchall()]
            if PN not in phonerecords:
                break
            else:
                print("\nPhone Number already Exists\n")
        while True:
            Email=input("\nEnter your Email ID : ")
            self.c.execute("Select Email_ID from user_details")
            print()
            emailrecords=[i[0] for i in self.c.fetchall()]
            if Email not in emailrecords:
                break
            else:
                print("\nEmail ID already Exists\n")
        name=input("\nEnter Your Name : ")
        while True:
            print("\n<---Password must have atleast 1 uppercase letter , 1 digit , 1 special character--->\n")
            passw = g.getpass("\nEnter your password : ")
            confirm_password=g.getpass("Re-enter your Password : ")
            p=validate_password(passw)
            if p==confirm_password:
                break
            else:
                print("\nPassword Not Match\n")
        self.c.execute("Insert into user_details (Phone_Number,Email_ID,Name,Password) values(%s,%s,%s,%s)",(PN,Email,name,passw))
        self.con.commit()
        print("\nUSER SIGNUP SUCCESSFULLY\n")
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    def login(self):
        def income():
            def add():
                print()
                IID=np.random.randint(10000,99999)
                Source=input("Enter your SOURCE of INCOME : ")
                Amount=float(input("Enter Your Salary Amount : "))
                DOI=date.today()
                self.c.execute("Insert into income (Income_ID,Income_Source,Salary_Amount,Income_Date) values (%s,%s,%s,%s)",(IID,Source,Amount,DOI))
                self.con.commit()
                print("\nIncome Details Added Successsfully")
            def view():
                self.c.execute("Select * from income")
                for i in self.c.fetchall():
                    print("\nIncome-ID        : ",i[0])
                    print("Source OF Income : ",i[1])
                    print("Salary Amount    : ",i[2])
                    print("Date of Income   : ",i[3])
                    print()
            def delete():
                IID=int(input("\nEnter Income-ID"))
                self.c.execute("delete from income where Income_ID = (%s)",(IID,))
                self.con.commit()
                print(f"\nINCOME-ID {IID} DETAILS DELETE Successfully ")
            while True:
                print("\nPress 1 for ADD INCOME")
                print("Press 2 for VIEW INCOME")
                print("Press 3 for Delete Income\n")
                x=int(input("Press a key : "))
                if x==1:
                    add()
                elif x==2:
                    view()
                elif x==3:
                    delete()
                else:
                    print("\nEXIT!\n")
                    break
        def expense():
            def upload():
                
                host = 'localhost'
                database = 'Expense_Tracker'
                a=input("\nPlease enter your Excel/CSV path: ")
                try:
                    a = a.strip('"')
                    a = a.replace("\\", "/")
                    df = pd.read_csv(a)
                except:
                    a = a.strip('"')
                    a = a.replace("\\", "/")
                    df = pd.read_excel(a)
                engine = create_engine(f'mysql+pymysql://{self.use}:{self.passwo}@{host}/{database}')
                df.to_sql('expenses', con=engine, if_exists='replace', index=False)
                print("\nYour File Upload Successfully\n")
            def manual():
                def add():
                    ID=np.random.randint(1000000,9999999)
                    d=date.today()
                    categ=input("\nEnter Expense Category : ")
                    desc=input("Enter Expense Description : ")
                    amnt=float(input("Enter Expense Amount : "))
                    P_M=input("Enter Your Payment Method : ")
                    crcy=input("Enter Your Currency : ")
                    self.c.execute("Insert into expenses (Expense_ID,Expense_Date,Category,Description,Amount,Payment_Method,Currency) values (%s,%s,%s,%s,%s,%s,%s)",(ID,d,categ,desc,amnt,P_M,crcy))
                    self.con.commit()
                    print("\nExpense Add Successfully\n")
                def view():
                    self.c.execute("select * from expenses")
                    data=self.c.fetchall()
                    print("Your expense data : ")
                    for i in data:
                        print("\n\tExpense-ID           : ",i[0])
                        print("\tExpense Date           : ",i[1])
                        print("\tExpense Category       : ",i[2])
                        print("\tExpense Discription    : ",i[3])
                        print("\tExpense Amount         : ",i[4])
                        print("\tExpense Payment Method : ",i[5])
                        print("\tExpense Currency       : ",i[6])
                        print()
                def delete():
                    ID=int(input("\nEnter Expense ID : "))
                    self.c.execute("delete from expenses where Expense_ID = (%s)",(ID,))
                    self.con.commit()
                    print("\nExpense Deleted Successfully\n")
                def convert():
                    df=pd.read_sql_query("Select * from expenses",self.con)
                    while True:
                        print("\nPress 1 for convert in Excel")
                        print("Press 2 for convert in CSV")
                        x=int(input("Press a key : "))
                        if x==1:
                            df.to_excel("Expenses.xlsx",index=False)
                            print("\nFile EXPORT Successfully\n")
                        elif x==2:
                            df.to_csv("Expenses.csv",index=False)
                            print("\nFile EXPORT Successfully\n")
                        else:
                            break
                print()
                self.c.execute("Create table if not exists expenses(Expense_Date date, Category text, Description text, Amount float, Payment_Method text, Currency text)")
                while True:
                    print("\nPress 1 for Add Expenses")
                    print("Press 2 for View Expenses")
                    print("Press 3 for Delete Expenses")
                    print("Press 4 for Export FILE TO CSV/Excel File")
                    x=int(input("\nPress a key : "))
                    if x==1:
                        add()
                    elif x==2:
                        view()
                    elif x==3:
                        delete()
                    elif x==4:
                        convert()
                    else:
                        print()
                        break
            while True:
                print("\nPress 1 for Upload Excel/CSV File")
                print("Press 2 for Update Expense Manually")
                x=int(input("Press a key : "))
                if x==1:
                    upload()
                elif x==2:
                    manual()
                else:
                    break
                    
        Email=input("\nEnter Your Email-ID : ")
        P_N=int(input("Enter your Phone Number : "))
        Passw=g.getpass("Enter Your Password : ")
        print()
        self.c.execute("Select Phone_Number,Email_ID,Password from user_details")
        data=self.c.fetchall()
        for i in data:
            if i[0]==P_N and i[1]==Email and i[2]==Passw:
                while True:
                    print("\nPress 1 for Income Details")
                    print("Press 2 for Expenses Details")
                    print("Press 3 for Exit!")
                    x=int(input("\nPress a key : "))
                    if x==1:
                        income()
                    elif x==2:
                        expense()
                    else:
                        print("\nWrong Input\n")
                        break
            else:
                print("\nWrong Credential\n")
################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
import mysql.connector
import pandas as pd
import re
import getpass as g
from datetime import date
import numpy as np
from sqlalchemy import create_engine
ET=Expense_Tracker()
print("\t\t\t\t\t\tWELCOME TO THE EXPENSE TRACKER\n")
print("The EXPENSE TRACKER work for you--------------->")
print("\nTrack all your Expense Here------>")
print("\nLets Get Started--->")
while True:
    print("\nPress 1 for Signup")
    print("Press 2 for Login")
    print("Press 3 for Exit!")
    x=int(input("Press a key : "))
    if x==1:
        ET.signup()
    elif x==2:
        ET.login()
    else:
        print("\nThanks\n")
        break


Enter your SQL Server UserName :  root
Enter your SQL Server Password :  ········


						WELCOME TO THE EXPENSE TRACKER

The EXPENSE TRACKER work for you--------------->

Track all your Expense Here------>

Lets Get Started--->

Press 1 for Signup
Press 2 for Login
Press 3 for Exit!


Press a key :  3



Thanks

